## Ответы по медицине

In [1]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-sc46xgyu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-sc46xgyu
  Resolved https://github.com/huggingface/transformers.git to commit 02fe3af275f0ad935441b813ef2e58b94d09788a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7160302 sha256=003c877f49bdbde1c1f3beec66fa9e6e0d6c40f813d3069da6f938ced0601a43
  Stored in directory: /tmp/pip-ephem-wheel-cache-171f44no/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2
  Cloning https://github.com

Загрузим датасет для обучения модели

In [2]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
from datasets import load_dataset

dataset = load_dataset("blinoff/medical_qa_ru_data", split='train[:5%]')
# не хватает RAM

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset medical_qa_ru_data downloaded and prepared to /root/.cache/huggingface/datasets/blinoff___medical_qa_ru_data/default/0.0.0/551476392fb5023cebc55b480ef3948c0ad2287c0ef0ad158a56fbb0bc37783f. Subsequent calls will reuse this data.


In [4]:
train_q, test_q, train_ans, test_ans = train_test_split(pd.Series(dataset['desc']),
                                                        pd.Series(dataset['ans']),
                                                        test_size=.2,
                                                        random_state=42)
del(dataset)

In [24]:
train_q.sample(2)

5072    У меня еще месяц назад это началось сначала на...
1801    С целью снизить либидо произвёл себе инъекцию ...
dtype: object

In [6]:
device = torch.device("cuda")

Будем использовать предобученную модель от Сбера

In [7]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Простой препроцессинг и разметка Вопрос-Ответ. (nx-ny)

In [8]:
def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [9]:
train_data = build_data(train_q, train_ans)
test_data = build_data(test_q, test_ans)

In [10]:
train_data[:3]

['\nx:Добрый день! Был незащищенный оральный секс с непроверенной девушкой. (Она делала минет без средств защиты). Спустя примерно полтора часа я обработал член хлоргексидином, а также сходил в аптеку и купил Сафоцид, выпил все 4 таблетки как того требует инструкция. На текущий момент прошло уже 4 дня, никаких видимых симптомов и отклонений не наблюдается. Скажите, высока ли вероятность подцепить какую-то заразу? Стоит ли пить антибиотики еще раз спустя неделю? Стоит ли воздерживаться от половых связей и какой срок?\ny:Если нет отклонений - то принимать антибиотики через неделю не имеет смысла.При половых контактах лучше применять барьерные способы защиты (презерватив).Анализы можете сдать на вич(не ранее,чем через 1-3 месяца),гепатиты(через 4-5 недель) и сифилис(6-7 недель) с момента предполагаемого заражения.; Нужно ли пока воздерживаться от связи с постоянным партнером?; Можете использовать презерватив.; Антибиотики принимать не нужно, пока нет точных данных о наличии какого-либо за

Токенизируем train и test

In [11]:
tokenizer.pad_token = tokenizer.eos_token

In [12]:
train_encodings = tokenizer(train_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

In [13]:
import pickle
with open('train_encodings.pkl', 'wb') as f:
   pickle.dump(train_encodings, f)
with open('test_encodings.pkl', 'wb') as f:
   pickle.dump(test_encodings, f)

In [14]:
from IPython.display import FileLink
FileLink(r'train_encodings.pkl')

/kaggle/working/train_encodings.pkl

In [15]:
FileLink(r'test_encodings.pkl')

/kaggle/working/test_encodings.pkl

In [16]:
# !pip install gdown

In [17]:
# !gdown 'https://drive.google.com/uc?id=1-4qmZECvqdl9lZ6RCnC86DL4KRo9k_im'

In [18]:
# !gdown 'https://drive.google.com/uc?id=1JoT5G942wYJ_sSyqwWQEQ3ktiNnhC0vv'

In [19]:
import pickle
with open('train_encodings.pkl', 'rb') as f:
  train_encodings = pickle.load(f)
with open('test_encodings.pkl', 'rb') as f:
  test_encodings = pickle.load(f)

Создадим датасет для PyTorch

In [20]:
class QAMedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = QAMedicalDataset(train_encodings)
test_dataset = QAMedicalDataset(test_encodings)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Зададим аргументы для обучения модели

In [21]:
training_args = TrainingArguments(
    output_dir="./qa_medical", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    report_to="none"
    )

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [23]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_29/2940101178.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.467300
1000,3.348000
1500,3.282500
2000,3.190900
2500,2.991200
3000,2.954700
3500,2.939900
4000,2.859100
4500,2.746400
5000,2.748700


/tmp/ipykernel_29/2940101178.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_29/2940101178.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_29/2940101178.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_29/2940101178.py:6: UserWarning: To copy construct from a 

TrainOutput(global_step=5712, training_loss=3.0128815381132923, metrics={'train_runtime': 1909.3689, 'train_samples_per_second': 11.962, 'train_steps_per_second': 2.992, 'total_flos': 5967648718848000.0, 'train_loss': 3.0128815381132923, 'epoch': 3.0})

In [25]:
tokenizer.save_pretrained('tokenizer_qa_medical_gen')
model.save_pretrained('model_qa_medical_gen')

In [28]:
tokenizer = AutoTokenizer.from_pretrained('tokenizer_qa_medical_gen')
model = AutoModelForCausalLM.from_pretrained('model_qa_medical_gen')

In [31]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=1,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = re.findall(r'\ny:(.+)', decoded)[-1] 
    return result.strip()

In [32]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'Стоп':
        break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
 Очень щекотно
Здравствуйте сделайте ванночки с отваром трав(укроп,чеснок) + настой ромашки; Омез на основе смекты (продаётся во многих аптеках ) 2 столовые ложки измельчают и заливают 200мл кипяченой воды. Принимают по 1 чайной ложке 3 раза за приём 20 минут до еды 4-5 раз.; Обратитесь к дерматологу для обследования или самостоятельно покажитесь врачу! Будьте здоровы! Возможны проблемы со щитовидкой!!! Исключить нужно все причины : ожирение, сахарный диабет,, гормоноэнтерит.Проверьте
 Пятки в разные стороны
Какие жалобы? Покажите больного гинекологу очно. Диагноз поставит врач после осмотра вашего здоровья! С уважением Галина Викторовна П., 59 лет, г Краснодар У меня такая проблема что постоянно тянущие стопы я очень худой сейчас ношу обувь не могу ходить тяжело только если присесть то болит живот спина боль чувствуется когда сяду потом отпустит Потом проходит а ноги так болю Вот как тогда быть? Я хочу попробывать мази какие нибудь но месячных боюсь